The purpose of this model is to create a very basic recommendation system based on the ratings the users gave to some movies.

In [28]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import KFold


As this is a 'custom' data set, there's the need to define a Reader object for Surprise to be able to parse the file or dataframe.

In [30]:
#Reading the file using pandas
rating = pd.read_csv('transformed_data/rating.csv')

In [32]:
#In the rating dataset, each line has the following format: 
#'id, user_id, rating, timestamp, rating_year'
#To read from dataframe we just need 'user_id', 'id', 'rating'.
rating_reader = Reader(rating_scale = (1, 5))
rating_data = Dataset.load_from_df(rating[['user_id', 'id', 'rating']], reader = rating_reader)


In [33]:
#Define a cross-validation iterator
kf = KFold(n_splits=5)

In [34]:
#Using the SVD algorithm.
algorithm = SVD()

In [35]:
#Let's train the algorithm.
for rating_trainset, rating_testset in kf.split(rating_data):

    #train and test.
    algorithm.fit(rating_trainset)
    rating_predictions = algorithm.test(rating_testset)

    #print Root Mean Squared Error
    accuracy.rmse(rating_predictions, verbose = True)

RMSE: 1.0022
RMSE: 1.0024
RMSE: 1.0026
RMSE: 1.0009
RMSE: 1.0025


Predict ratings

In [36]:
#Let's get some user id with known rating and try to predict its rating.
user_id = 14
id = 'as1577'
predict = algorithm.predict(user_id, id, r_ui = 4, verbose = True)

user: 14         item: as1577     r_ui = 4.00   est = 3.68   {'was_impossible': False}
